# TensorFlow Estimators

```{admonition} Attribution
This notebook builds on Chapter 14: *Going Deeper – The Mechanics of TensorFlow* of {cite}`RaschkaMirjalili2019`.
```

The `tf.estimator` API
encapsulates the underlying steps in machine learning tasks, such as training,
inference, and evaluation. Estimators are more encapsulated but also
more scalable when compared to the previous approaches that we have covered
in the previous notebook. Also, the `tf.estimator` API adds support for running models on
multiple platforms without requiring major code changes, which makes them more
suitable for the so-called "production phase" in industry applications. 

In addition,
TensorFlow comes with a selection of off-the-shelf estimators for common machine
learning and deep learning architectures that are useful for comparison studies, for
example, to quickly assess whether a certain approach is applicable to a particular
dataset or problem.

## Working with feature columns

Note that while numeric data can be either continuous or discrete,
in the context of the TensorFlow API, "numeric" data specifically refers to continuous
data of the floating point type. Sometimes, feature sets are comprised of a mixture of different feature types. While TensorFlow Estimators were designed to handle all these different types of features, we must specify how each feature should be interpreted by the Estimator. For example, consider a scenario with a set of seven different features, as shown in the following figure:

The features shown in the figure (model year, cylinders, displacement, horsepower, weight, acceleration, and origin) were obtained from the Auto MPG dataset, which is a common machine learning benchmark dataset for predicting the fuel efficiency of a car in miles per gallon (MPG). The full dataset and its description are available from UCI's machine learning repository at https://archive.ics.uci.edu/ml/ datasets/auto+mpg.